In [1]:
import os

In [2]:
%pwd

'/Users/inamgurung/Desktop/Projects/AWS/Kidney-Disease-Classification/Kidney-Disease-Classification-Deep-Learning-Project/research'

In [3]:
os.chdir("../") # change directory to project root folder

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_inclue_top: bool
    params_weights: str
    params_classes: int

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        prepare_base_model_config = self.config.prepare_base_model

        create_directories([prepare_base_model_config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(prepare_base_model_config.root_dir),
            base_model_path = Path(prepare_base_model_config.base_model_path),
            updated_base_model_path = Path(prepare_base_model_config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_inclue_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config



In [7]:
import os 
import urllib.request
from zipfile import ZipFile
import tensorflow

2026-01-19 16:08:09.983456: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
            self.model = tensorflow.keras.applications.VGG16(
                input_shape=self.config.params_image_size,
                weights=self.config.params_weights,
                include_top=self.config.params_inclue_top
            )

            self.save_model(self.model, self.config.base_model_path)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tensorflow.keras.layers.Flatten()(model.output)
        prediction = tensorflow.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tensorflow.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            loss=tensorflow.keras.losses.CategoricalCrossentropy(),
            optimizer=tensorflow.keras.optimizers.SGD(learning_rate=learning_rate),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(full_model, self.config.updated_base_model_path)
    
    @staticmethod
    def save_model(model: tensorflow.keras.Model, model_path: Path):
         model.save(model_path)
            

In [9]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2026-01-19 16:08:16,076 - INFO - yaml file: config/config.yaml loaded successfully]
[2026-01-19 16:08:16,086 - INFO - yaml file: params.yaml loaded successfully]
[2026-01-19 16:08:16,099 - INFO - Created directory at: artifacts]
[2026-01-19 16:08:16,101 - INFO - Created directory at: artifacts/prepare_base_model]
[2026-01-19 16:08:16,819 - WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                  